In [12]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools
from datetime import datetime

In [13]:
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [14]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [15]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        #mse=mean_squared_error(y, y_predict, squared=False)
        acc=correct/len(y_predict)
        print("Accuracy",correct/len(y_predict))
        #print("Errore quadratico medio: ", mse)
        return acc
    
    def accuracy(self, X, y):
        y_predict=self.predict(X)
        correct = np.sum(y_predict == y)
        acc=correct/len(y_predict)
        return acc
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))
    

In [16]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
#err = []
acc = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
print(datetime.now())
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    print("\nBest model", best_C, best_sigma)
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    best_model.fit(X_train, y_train)
    #err.append(best_model.score(X_validation, y_validation))
    acc.append(best_model.score(X_validation, y_validation))
print(datetime.now())    

2020-10-03 17:11:20.422798
     pcost       dcost       gap    pres   dres
 0: -1.4002e+01 -4.7754e+00  5e+02  2e+01  3e-16
 1: -2.7794e+00 -1.8569e+00  4e+01  2e+00  4e-16
 2: -3.9341e-01 -1.6171e+00  2e+00  4e-02  7e-16
 3: -3.1655e-01 -4.7644e-01  2e-01  1e-16  1e-15
 4: -3.1873e-01 -3.2061e-01  2e-03  2e-17  3e-16
 5: -3.1887e-01 -3.1894e-01  7e-05  1e-16  2e-16
 6: -3.1888e-01 -3.1888e-01  2e-06  4e-17  2e-16
 7: -3.1888e-01 -3.1888e-01  3e-08  2e-17  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4002e+01 -4.7754e+00  5e+02  2e+01  3e-16
 1: -2.7794e+00 -1.8569e+00  4e+01  2e+00  4e-16
 2: -3.9341e-01 -1.6171e+00  2e+00  4e-02  7e-16
 3: -3.1655e-01 -4.7644e-01  2e-01  1e-16  1e-15
 4: -3.1873e-01 -3.2061e-01  2e-03  2e-17  3e-16
 5: -3.1887e-01 -3.1894e-01  7e-05  1e-16  2e-16
 6: -3.1888e-01 -3.1888e-01  2e-06  4e-17  2e-16
 7: -3.1888e-01 -3.1888e-01  3e-08  2e-17  2e-16
Optimal solution found.
0 out of 24 predictions correct
Accuracy 0.0


 3: -7.0362e-01 -8.3837e-01  1e-01  8e-17  1e-15
 4: -7.1015e-01 -7.2997e-01  2e-02  3e-17  4e-16
 5: -7.1290e-01 -7.1614e-01  3e-03  3e-17  4e-16
 6: -7.1386e-01 -7.1470e-01  8e-04  9e-17  3e-16
 7: -7.1413e-01 -7.1429e-01  2e-04  1e-16  3e-16
 8: -7.1419e-01 -7.1422e-01  3e-05  1e-16  4e-16
 9: -7.1420e-01 -7.1420e-01  6e-07  2e-16  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.4301e+00 -2.8852e+00  3e+02  2e+01  4e-16
 1: -1.8149e+00 -2.4539e+00  1e+01  7e-01  5e-16
 2: -8.3865e-01 -1.6399e+00  2e+00  4e-02  1e-15
 3: -7.0362e-01 -8.3837e-01  1e-01  8e-17  1e-15
 4: -7.1015e-01 -7.2997e-01  2e-02  3e-17  4e-16
 5: -7.1290e-01 -7.1614e-01  3e-03  3e-17  4e-16
 6: -7.1386e-01 -7.1470e-01  8e-04  9e-17  3e-16
 7: -7.1413e-01 -7.1429e-01  2e-04  1e-16  3e-16
 8: -7.1419e-01 -7.1422e-01  3e-05  1e-16  4e-16
 9: -7.1420e-01 -7.1420e-01  6e-07  2e-16  4e-16
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost   

     pcost       dcost       gap    pres   dres
 0: -2.6359e+01 -2.2041e+00  6e+02  3e+01  2e-15
 1: -6.5441e-01 -2.0054e+00  9e+00  3e-01  2e-15
 2: -3.4118e-01 -1.2865e+00  1e+00  1e-02  7e-16
 3: -3.1953e-01 -3.5143e-01  3e-02  4e-05  2e-15
 4: -3.1996e-01 -3.2033e-01  4e-04  5e-07  4e-16
 5: -3.1997e-01 -3.2002e-01  4e-05  5e-08  3e-16
 6: -3.1997e-01 -3.2002e-01  4e-05  5e-08  4e-16
 7: -3.1998e-01 -3.2001e-01  3e-05  3e-08  4e-16
 8: -3.1998e-01 -3.2000e-01  2e-05  2e-08  3e-16
 9: -3.1998e-01 -3.1999e-01  1e-05  8e-09  3e-16
10: -3.1999e-01 -3.1999e-01  4e-06  3e-09  4e-16
11: -3.1999e-01 -3.1999e-01  2e-06  2e-09  3e-16
12: -3.1999e-01 -3.1999e-01  1e-06  6e-10  4e-16
13: -3.1999e-01 -3.1999e-01  6e-07  3e-10  3e-16
14: -3.1999e-01 -3.1999e-01  6e-08  2e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6359e+01 -2.2041e+00  6e+02  3e+01  2e-15
 1: -6.5441e-01 -2.0054e+00  9e+00  3e-01  2e-15
 2: -3.4118e-01 -1.2865e+00  1e+00  1e-02  7e-1

 2: -4.9440e-01 -1.3694e+00  1e+00  6e-03  6e-16
 3: -4.7968e-01 -4.9296e-01  1e-02  4e-05  1e-15
 4: -4.8000e-01 -4.8013e-01  1e-04  4e-07  4e-16
 5: -4.8000e-01 -4.8000e-01  3e-06  8e-09  6e-16
 6: -4.8000e-01 -4.8000e-01  7e-07  2e-09  4e-16
 7: -4.8000e-01 -4.8000e-01  2e-07  3e-10  3e-16
Optimal solution found.
8 out of 24 predictions correct
Accuracy 0.3333333333333333
     pcost       dcost       gap    pres   dres
 0: -4.7112e+01 -1.9943e+00  4e+02  2e+01  3e-15
 1: -1.1493e+00 -1.9790e+00  5e+00  2e-01  3e-15
 2: -7.5208e-01 -1.4187e+00  7e-01  3e-17  6e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  1e-16  7e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  2e-16  5e-16
 5: -8.0000e-01 -8.0001e-01  7e-06  4e-17  1e-15
 6: -8.0000e-01 -8.0000e-01  1e-06  6e-17  4e-16
 7: -8.0000e-01 -8.0000e-01  7e-07  8e-17  7e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7112e+01 -1.9943e+00  4e+02  2e+01  3e-15
 1: -1.1493e+00 -1.9790e+00  5e+00  2e-01  3e-15
 2: -7.5208e

     pcost       dcost       gap    pres   dres
 0: -3.6360e+01 -2.0412e+00  6e+02  2e+01  3e-15
 1: -8.9286e-01 -2.0298e+00  7e+00  3e-01  3e-15
 2: -4.9441e-01 -1.3695e+00  1e+00  6e-03  7e-16
 3: -4.7968e-01 -4.9297e-01  1e-02  4e-05  2e-15
 4: -4.8000e-01 -4.8013e-01  1e-04  4e-07  4e-16
 5: -4.8000e-01 -4.8000e-01  1e-06  4e-09  5e-16
 6: -4.8000e-01 -4.8000e-01  1e-08  4e-11  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.6360e+01 -2.0412e+00  6e+02  2e+01  3e-15
 1: -8.9286e-01 -2.0298e+00  7e+00  3e-01  3e-15
 2: -4.9441e-01 -1.3695e+00  1e+00  6e-03  7e-16
 3: -4.7968e-01 -4.9297e-01  1e-02  4e-05  2e-15
 4: -4.8000e-01 -4.8013e-01  1e-04  4e-07  4e-16
 5: -4.8000e-01 -4.8000e-01  1e-06  4e-09  5e-16
 6: -4.8000e-01 -4.8000e-01  1e-08  4e-11  4e-16
Optimal solution found.
8 out of 24 predictions correct
Accuracy 0.3333333333333333
     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+

...

     pcost       dcost       gap    pres   dres
 0:  1.5551e+08 -7.2891e+08  1e+09  3e-02  4e-12
 1:  4.1191e+07 -6.0118e+07  1e+08  1e-03  5e-11
 2:  6.2111e+06 -6.8413e+06  1e+07  4e-12  2e-12
 3:  8.9144e+05 -9.9373e+05  2e+06  2e-13  1e-12
 4:  1.2751e+05 -1.4167e+05  3e+05  2e-13  4e-13
 5:  1.8152e+04 -2.0396e+04  4e+04  5e-14  1e-13
 6:  2.5534e+03 -2.9685e+03  6e+03  5e-14  5e-14
 7:  3.4701e+02 -4.4384e+02  8e+02  3e-14  2e-14
 8:  4.1822e+01 -7.1132e+01  1e+02  4e-15  7e-15
 9:  2.1818e+00 -1.3625e+01  2e+01  2e-15  2e-15
10: -2.1207e+00 -4.0290e+00  2e+00  1e-15  9e-16
11: -2.4977e+00 -2.9577e+00  5e-01  3e-16  3e-16
12: -2.6420e+00 -2.7862e+00  1e-01  2e-15  3e-16
13: -2.7020e+00 -2.7418e+00  4e-02  1e-15  3e-16
14: -2.7145e+00 -2.7170e+00  3e-03  1e-15  3e-16
15: -2.7154e+00 -2.7154e+00  3e-05  6e-16  3e-16
16: -2.7154e+00 -2.7154e+00  3e-07  1e-15  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.5551e+08 -7.2891e+08  1e+09  3e-02  4e-1

     pcost       dcost       gap    pres   dres
 0:  4.9231e+08 -4.0884e+09  7e+09  2e-01  1e-11
 1:  2.4646e+08 -3.3461e+08  6e+08  9e-03  3e-09
 2:  3.8912e+07 -4.9616e+07  9e+07  3e-12  8e-12
 3:  5.6597e+06 -6.5123e+06  1e+07  9e-12  3e-12
 4:  8.1208e+05 -9.1436e+05  2e+06  4e-13  2e-12
 5:  1.1560e+05 -1.3091e+05  2e+05  6e-13  5e-13
 6:  1.6212e+04 -1.9064e+04  4e+04  2e-13  2e-13
 7:  2.1786e+03 -2.8696e+03  5e+03  2e-13  8e-14
 8:  2.4892e+02 -4.6968e+02  7e+02  3e-14  3e-14
 9:  1.5974e+01 -1.2663e+02  1e+02  2e-15  1e-14
10: -2.5624e+01 -8.5860e+01  6e+01  1e-14  7e-15
11: -3.2212e+01 -9.0310e+01  6e+01  3e-14  1e-14
12: -5.4860e+01 -6.7456e+01  1e+01  3e-14  1e-14
13: -6.0502e+01 -6.0895e+01  4e-01  9e-15  1e-14
14: -6.0717e+01 -6.0733e+01  2e-02  3e-14  1e-14
15: -6.0725e+01 -6.0725e+01  2e-04  8e-15  1e-14
16: -6.0725e+01 -6.0725e+01  2e-06  3e-14  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  4.9231e+08 -4.0884e+09  7e+09  2e-01  1e-1

24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -6.2453e+04 -1.2022e+09  1e+09  3e-11  3e-11
 1: -6.2538e+04 -1.2430e+07  1e+07  3e-11  3e-11
 2: -7.0082e+04 -5.1744e+05  4e+05  5e-11  3e-11
 3: -1.1906e+05 -4.1616e+05  3e+05  1e-11  2e-11
 4: -1.2425e+05 -4.1309e+05  3e+05  6e-11  2e-11
 5: -1.4537e+05 -3.9180e+05  2e+05  4e-11  2e-11
 6: -1.6751e+05 -3.2123e+05  2e+05  2e-11  2e-11
 7: -1.8530e+05 -2.7731e+05  9e+04  4e-11  3e-11
 8: -1.9980e+05 -2.4840e+05  5e+04  2e-11  2e-11
 9: -2.1065e+05 -2.3145e+05  2e+04  5e-11  3e-11
10: -2.1591e+05 -2.2308e+05  7e+03  9e-11  3e-11
11: -2.1836e+05 -2.2008e+05  2e+03  2e-11  3e-11
12: -2.1914e+05 -2.1918e+05  4e+01  6e-11  3e-11
13: -2.1916e+05 -2.1916e+05  8e-01  9e-11  3e-11
14: -2.1916e+05 -2.1916e+05  8e-03  4e-11  3e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.2453e+04 -1.2022e+09  1e+09  3e-11  3e-11
 1: -6.2538e+04 -1.2430e+07  1e+07  3e-11  3e-11
 

     pcost       dcost       gap    pres   dres
 0: -3.5706e+05 -1.2020e+09  1e+09  3e-11  2e-11
 1: -3.5713e+05 -1.2495e+07  1e+07  2e-10  3e-11
 2: -3.6369e+05 -5.9951e+05  2e+05  1e-10  3e-11
 3: -4.6548e+05 -4.8244e+05  2e+04  5e-11  4e-11
 4: -4.6759e+05 -4.8249e+05  1e+04  8e-11  3e-11
 5: -4.7187e+05 -4.8183e+05  1e+04  6e-11  3e-11
 6: -4.7559e+05 -4.7820e+05  3e+03  6e-11  3e-11
 7: -4.7672e+05 -4.7740e+05  7e+02  2e-11  4e-11
 8: -4.7688e+05 -4.7729e+05  4e+02  2e-11  3e-11
 9: -4.7704e+05 -4.7717e+05  1e+02  2e-11  3e-11
10: -4.7708e+05 -4.7713e+05  5e+01  3e-11  3e-11
11: -4.7711e+05 -4.7711e+05  3e+00  2e-11  3e-11
12: -4.7711e+05 -4.7711e+05  9e-02  3e-11  3e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.5706e+05 -1.2020e+09  1e+09  3e-11  2e-11
 1: -3.5713e+05 -1.2495e+07  1e+07  2e-10  3e-11
 2: -3.6369e+05 -5.9951e+05  2e+05  1e-10  3e-11
 3: -4.6548e+05 -4.8244e+05  2e+04  5e-11  4e-11
 4: -4.6759e+05 -4.8249e+05  1e+04  8e-11  3e-1

24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -4.6667e+05 -6.6865e+08  7e+08  4e-11  2e-11
 1: -4.6714e+05 -7.4788e+06  7e+06  3e-11  4e-11
 2: -5.0800e+05 -8.6730e+05  4e+05  3e-11  4e-11
 3: -7.9697e+05 -8.2632e+05  3e+04  4e-11  6e-11
 4: -7.9982e+05 -8.0047e+05  6e+02  3e-11  7e-11
 5: -7.9988e+05 -7.9996e+05  8e+01  4e-10  6e-11
 6: -7.9990e+05 -7.9991e+05  1e+01  3e-11  6e-11
 7: -7.9991e+05 -7.9991e+05  2e+00  4e-11  6e-11
 8: -7.9991e+05 -7.9991e+05  7e-01  3e-10  6e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.6667e+05 -6.6865e+08  7e+08  4e-11  2e-11
 1: -4.6714e+05 -7.4788e+06  7e+06  3e-11  4e-11
 2: -5.0800e+05 -8.6730e+05  4e+05  3e-11  4e-11
 3: -7.9697e+05 -8.2632e+05  3e+04  4e-11  6e-11
 4: -7.9982e+05 -8.0047e+05  6e+02  3e-11  7e-11
 5: -7.9988e+05 -7.9996e+05  8e+01  4e-10  6e-11
 6: -7.9990e+05 -7.9991e+05  1e+01  3e-11  6e-11
 7: -7.9991e+05 -7.9991e+05  2e+00  4e-11  6e-11
 

In [17]:
print(np.mean(acc))

1.0


In [18]:
print(acc)

[1.0, 1.0, 1.0, 1.0, 1.0]
